In [1]:
import os
import sys
import numpy as np
import pandas as pd
from gnnwr.datasets import init_dataset
from gnnwr.models import GTNNWR

In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd

data = pd.read_csv("/Users/cypei/Downloads/NE_glb_cities_era5_ae_rad_1213_agg.csv") #E:\NE_glb_cities_era5_ae_0832.csv")
data.columns
print(len(data))

data = data.dropna(subset=['longitude','latitude',
    'LAI','emission', 'volumetric_soil_water_layer_1',
    'Solar_Radiation_Flux','u_component_of_wind_10m',
    'v_component_of_wind_10m'
])
print(len(data))

232862
229382


In [5]:
data['localtime'] = data['localtime'].str.extract(r'(^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})')

data['localtime'] = pd.to_datetime(data['localtime'])

data['timestamp'] = (data['localtime'].astype('int64') // 10**9)//3600 

data['timestamp'] = data['timestamp'] - data['timestamp'].min()


In [7]:
# Randomly sample 1/10 of the dataset
data = data.sample(frac=0.1, random_state=42)


In [27]:
os.chdir('/Users/cypei/Downloads/gnnwr/')

In [39]:
from sklearn.preprocessing import StandardScaler

data["id"] = np.arange(len(data))

def standardize_columns(data, columns):
    # 使用StandardScaler进行标准化
    scaler = StandardScaler()
    data[columns] = scaler.fit_transform(data[columns])
    return data, scaler

# 标准化x_column中的数据
data, scaler = standardize_columns(data, 
                                   ['LAI', 'emission', 'volumetric_soil_water_layer_1',
                                    'Solar_Radiation_Flux', 'u_component_of_wind_10m', 
                                    'v_component_of_wind_10m'])



train_dataset, val_dataset, test_dataset = init_dataset(data=data,
                                                        test_ratio=0.15,
                                                        valid_ratio=0.1,
                                                        x_column=['LAI','emission', 'volumetric_soil_water_layer_1',
                                                                  'Solar_Radiation_Flux','u_component_of_wind_10m',
                                                                  'v_component_of_wind_10m'],
                                                        y_column=['NE'],
                                                        spatial_column=['longitude', 'latitude'],
                                                        temp_column=['timestamp'],
                                                        id_column=['id'],
                                                        use_model="gtnnwr",
                                                        sample_seed=48,
                                                        batch_size=256)
optimizer_params = {
    "scheduler":"MultiStepLR",
    "scheduler_milestones":[1000, 2000, 3000, 4000],
    "scheduler_gamma":0.8,
}
log_path = '../gtnnwr_runs'
gtnnwr = GTNNWR(train_dataset, val_dataset, test_dataset, [[3], [2048, 512, 256, 32]],optimizer='Adam')
gtnnwr.add_graph

#gtnnwr.run(200,10)
gtnnwr.run(500,80,10)

TypeError: GNNWR.run() takes from 1 to 3 positional arguments but 4 were given

In [23]:
import os

# 获取相对路径的完整路径
log_dir = os.path.abspath('../gtnnwr_logs')
print(log_dir)

/Users/gtnnwr_logs


In [ ]:
gtnnwr.result()

In [ ]:
gtnnwr.reg_result('./gtnnwr_result.csv')